In [1]:
import cvxpy as cp
import numpy as np

In [2]:
# define variables
x = cp.Variable()
y = cp.Variable()

In [15]:
# help(cp.Variable)

In [3]:
x

Variable((), var1)

In [4]:
y

Variable((), var2)

In [5]:

# Define the objective function
objective = cp.Minimize(x**2 + y**2)
objective

Minimize(Expression(CONVEX, NONNEGATIVE, ()))

In [6]:
# Define constraints
constraints = [x + y == 1, x - y >= 1]
constraints

[Equality(Expression(AFFINE, UNKNOWN, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Inequality(Constant(CONSTANT, NONNEGATIVE, ()))]

In [7]:
# Formulate and solve the problem
problem = cp.Problem(objective, constraints)
problem.solve()

1.0000000000000004

In [8]:
# Print results
print("Optimal value:", problem.value)
print("Optimal x:", x.value)
print("Optimal y:", y.value)

Optimal value: 1.0000000000000004
Optimal x: 1.0000000000000002
Optimal y: 3.9305419627733755e-23


In [9]:
x = cp.Variable()
y = cp.Variable()

# Objective
objective = cp.Maximize(3*x + 2*y)

# Constraints
constraints = [x + y <= 4, x - 2*y >= -1, x >= 0, y >= 0]

# Formulate and solve
problem = cp.Problem(objective, constraints)
problem.solve()

# Display results
print("Optimal value:", problem.value)
print("Optimal x:", x.value)
print("Optimal y:", y.value)


Optimal value: 11.999999995678937
Optimal x: 3.9999999986829056
Optimal y: -1.8488909965547973e-10


#### **Vectors and matrices**
* `Variables` can be scalars, vectors, or matrices, meaning they are 0, 1, or 2 dimensional.

In [10]:
# A scalar variable
a = cp.Variable()

# Vector variable with shape (5, )
X = cp.Variable(5)

# Matrix variable with shape (5, 1)
W = cp.Variable((5, 1))

# Matrix variable with shape (7, 5)
Q = cp.Variable((7, 5))

R = Q @ W
R

Expression(UNKNOWN, UNKNOWN, (7, 1))

In [11]:
R.shape

(7, 1)

In [12]:
X.shape

(5,)

In [13]:
c = X @ W
c.shape

(1,)

In [14]:
c.value

In [15]:
R.value

In [16]:
R.domain

[]

In [17]:
R.T.shape

(1, 7)

##### **Solves a bounded least-squares problem**

In [18]:
import cvxpy as cp
import numpy as np

In [20]:
# We aim to solve the optimization problem:
#       minimize || A.x - b ||^2 ; s.t constraint 0 <= x <= 1
#           x

#       A => R(mXn), x => R(n), b => R(m)

# In mathematical terms, we can write this problem in convex optimization form:
#       minimize f(x) = (A.x - b)^T (A.x - b)
#           x

#       subject to: 0 <= x(i) <= 1  for all i

# Minimizing this term -> || A.x - b ||^2, makes A.x as close as possible to b


m = 16
n = 8

np.random.seed(42)
A = np.random.randn(m, n) * 2.0
b = np.random.rand(m) 

# let's construct the problem.

x = cp.Variable(n)

# Objective function: minimize ||A.x - b||^2
objective = cp.Minimize(cp.sum_squares(A @ x - b))

# Constraints
constraints = [x >= 0, x <= 1]

# Define the problem and solve
problem = cp.Problem(objective, constraints)

print("Optimal value:", problem.solve())
print("Optimal var:")
print(x.value)  # A numpy ndarray.

Optimal value: 4.124160456449218
Optimal var:
[ 2.10263212e-02  1.04441002e-01  7.46439739e-02 -2.49595424e-19
  1.15712700e-19 -3.08139871e-20 -2.01204900e-19 -1.38076104e-19]


In [ ]:
## Solve the above problem using scipy.optimize
from scipy.optimize import minimize


m = 16
n = 8

np.random.seed(42)
A = np.random.randn(m, n) * 2.0
b = np.random.rand(m) 


# Objective function: || A.x - b || ^2
def objective(x):
    return np.sum((A @ x - b) ** 2)

# Bounds for each element of x: 0 <= x <= 1
bounds = [(0, 1) for _ in range(n)]

# Initial guess for x (can be zeros or any random guess within bounds)
x0 = np.zeros(n)

# Solve the problem 
result = minimize(objective, x0, bounds=bounds) 

# Display the results
print("Optimal value:", result.fun)
print("Optimal x:", result.x)

Optimal value: 4.124160456449236
Optimal x: [0.0210263  0.104441   0.07464397 0.         0.         0.
 0.         0.        ]


##### **Solves a Quadratic Programming Problem (QPP)**

In [65]:
# Objective Function:
#       Maximize f(a) = 1^T.a - 0.5 * (a^T.Q.a)
#           a
# Subject to: y^T.a = 0 and a(i) >= 0 for all i

m, n = 12, 4

# np.random.seed(47)

X = np.random.randn(m, n) * 2.0
y = np.random.choice([-1, 1], size=m)

# Compute Q matrix
Q = np.outer(y, y) * (X @ X.T)

# Define the variable alpha (a)
alpha = cp.Variable(m)

# Define the minimization objective (minimizing -f(alpha))
objective = cp.Minimize(-cp.sum(alpha) + 0.5 * cp.quad_form(alpha, Q))

# Define constraints
constraints = [alpha >= 0, y @ alpha == 0]

# Formulate and solve the problem
problem = cp.Problem(objective, constraints)
optimal_value = problem.solve()

# Display the results
print("Optimal value:", optimal_value)
print("Optimal alpha:", alpha.value)

DCPError: Problem does not follow DCP rules. Specifically:
The objective is not DCP. Its following subexpressions are not:
QuadForm(var784, [[23.95 0.11 ... 1.08 -3.03]
 [0.11 12.28 ... 7.22 9.48]
 ...
 [1.08 7.22 ... 5.33 5.11]
 [-3.03 9.48 ... 5.11 9.84]])

In [34]:
from scipy.optimize import minimize
import numpy as np

# let's try to solve the above using scipy.optimize

# Problem data
m = 16  # Number of samples (example)
n = 5   # Number of features (example)

# Randomly generated feature matrix and labels for illustration
np.random.seed(111)
X = np.random.randn(m, n)
y = np.random.choice([-1, 1], size=m)

# Compute Q matrix
Q = np.outer(y, y) * (X @ X.T)

# Define the objective function for the dual SVM
def objective(alpha):
    return -np.sum(alpha) + 0.5 * alpha @ Q @ alpha


# 1.Equality Constraint: y^T.alpha = 0
# 2.Inequality Constraint (Bounds): alpha >= 0

# In SciPy’s `minimize` function, equality constraints and bounds are handled separately. Let’s look at each in turn.

# Equality Constraint: y^T.alpha = 0
constraints = [
    {'type': 'eq', 'fun': lambda alpha: np.dot(y, alpha)}
]

# Bounds for each alpha: alpha >= 0 
bounds = [(0, None) for _ in range(m)]  

# Initial guess for alpha
alpha0 = np.zeros(m)

# Solve the problem
result = minimize(objective, alpha0, bounds=bounds, constraints=constraints)

# Display the results
print("Optimal value:", -result.fun)  # Negate the result because we minimized -f(alpha)
print("Optimal alpha:", result.x)


Optimal value: 762112.9913198742
Optimal alpha: [48607.42489041 54078.57256721 32314.1711677  15119.24907212
 73598.28044637 57274.07560826 34396.04200658 64947.6398479
 69093.52881099 19366.32611972 44840.62382719 52100.85528769
 41156.98116012 50516.15382827 65948.17581545 38910.87425279]


In [7]:
import numpy as np
from scipy.optimize import minimize

# Problem data
n = 16  # Number of samples (example)
d = 4   # Number of features (example)
C = 10.0  # Regularization parameter (upper bound for alpha)

# Randomly generated feature matrix and labels for illustration
np.random.seed(0)
X = np.random.randn(n, d)
y = np.random.choice([-1, 1], size=n)

# Compute the Gram matrix K
K = X @ X.T

# Objective function: Maximize f(alpha) = 1^T * alpha - 0.5 * (alpha * y)^T * K * (alpha * y)
def objective(alpha):
    alpha_y = alpha * y  # Element-wise product of alpha and y
    return -np.sum(alpha) + 0.5 * np.dot(alpha_y, K @ alpha_y)

# Equality constraint: alpha^T * y = 0
constraints = [
    {'type': 'eq', 'fun': lambda alpha: np.dot(y, alpha)}
]

# Bounds for each alpha: 0 <= alpha <= C
bounds = [(0, C) for _ in range(n)]

# Initial guess for alpha
alpha0 = np.zeros(n)

# Solve the problem using scipy.optimize.minimize
result = minimize(objective, alpha0, bounds=bounds, constraints=constraints, method='SLSQP')

# Display the results
print("Optimal value:", -result.fun)  # Negate the result because we maximized
print("Optimal alpha:", result.x)

Optimal value: 63.09258011944651
Optimal alpha: [5.66146425e+00 0.00000000e+00 0.00000000e+00 1.00000000e+01
 1.00000000e+01 9.30800982e-13 2.02821610e-12 2.36553088e+00
 2.51334964e+00 0.00000000e+00 4.19071700e+00 1.00000000e+01
 1.00000000e+01 1.00000000e+01 0.00000000e+00 0.00000000e+00]


In [25]:
import scipy

In [28]:
help(scipy.optimize.minimize)

Help on function minimize in module scipy.optimize._minimize:

minimize(fun, x0, args=(), method=None, jac=None, hess=None, hessp=None, bounds=None, constraints=(), tol=None, callback=None, options=None)
    Minimization of scalar function of one or more variables.
    
    Parameters
    ----------
    fun : callable
        The objective function to be minimized.
    
            ``fun(x, *args) -> float``
    
        where ``x`` is a 1-D array with shape (n,) and ``args``
        is a tuple of the fixed parameters needed to completely
        specify the function.
    x0 : ndarray, shape (n,)
        Initial guess. Array of real elements of size (n,),
        where ``n`` is the number of independent variables.
    args : tuple, optional
        Extra arguments passed to the objective function and its
        derivatives (`fun`, `jac` and `hess` functions).
    method : str or callable, optional
        Type of solver.  Should be one of
    
            - 'Nelder-Mead' :ref:`(see he